In [1]:
using DrWatson;
@quickactivate "NumericalAnalysis";

# Numerical Analysis Course Notes

This set of notes correspond to a first course in numerical analysis for undergraduates students of mathematics. In fact, the notes where developed to correspond to the course MATH 361 *Numerical Analysis* at the [University of Scranton](https://www.scranton.edu/) for the 2021 fall semester. The notes closely follow the textbook *Fundamentals of Numerical Computation* (FNC) by Driscoll and Braun. However, in the notes and in the course we will use the [Julia](https://julialang.org/) programming language to implement and explore various numerical methods. Note that there is a useful [companion website](https://tobydriscoll.net/project/fnc/) to the FNC book where one may find links to known [errors](https://github.com/tobydriscoll/fnc-extras/blob/master/errata/errata.md) as well as other [extras](https://github.com/tobydriscoll/fnc-extras) such as code and supplemental [lecture videos](https://www.youtube.com/playlist?list=PLvUvOH0OYx3BcZivtXMIwP6hKoYv0YvGn). An additional text resource that we may refer to from time to time is *A First Course in Numerical Methods* by Ascher and Greif, this book is recommended but not required and also has a corresponding [website](https://archive.siam.org/books/cs07/) with useful extra materials.

Rest assured that this course does not require a deep knowledge of computer science or extensive programming experience. An introduction to Julia appropriate for this course may be found in the Julia for Numerical Analysis [notebook](https://github.com/jmgraham30/MATH361/blob/main/NumericalJuliaIntro.ipynb). By going through this notebook, carefully reading the example code presented in class, and asking questions you will pick up all the programming you need to know for the course. If you want a broader introduction to Julia, we recommend following the [Introduction to Julia](https://juliaacademy.com/p/intro-to-julia) lesson which is part of the [Julia Academy](https://juliaacademy.com/) series.  There are also many other good Julia tutorials to be found online via a simple google search.  

# What is Numerical Analysis?

[Scientific computing](https://en.wikipedia.org/wiki/Computational_science#:~:text=Computational%20science%2C%20also%20known%20as,understand%20and%20solve%20complex%20problems.) is a scientific discipline that seeks to develop and implement [algorithms](https://en.wikipedia.org/wiki/Algorithm) that may be used to address mathematical problems that arise in our attempts to study the world around us. Scientic computing is routinely involved in all of the natural sciences, engineering, economics and finance, machine learning, etc. [Numerical analysis](https://en.wikipedia.org/wiki/Numerical_analysis) is the mathematical theory behind the algorithms of scientific computing. Thus, numerical analysis (somtimes called numerical mathematics) is a **branch of mathematics** that provides the mathematical foundations for scientific computing.   

Scientific computing arises as a result of the complexity of the natural world. Consider as an example the problem of weather forcasting. Weather is the result of entirely physical processes that determine the state of the atmosphere, and weather prediction is concerned with predicting how the atmosphere will evolve over a given period of time. That is, we want to input the current atmospheric conditions and output what the atmospheric conditions will be over some future time period. The atmosphere is a fluid with three velocity component variables $u$, $v$, and $w$ as well as a pressure $p$, density $\rho$, temperature $T$, and specific hunidity $q$. Each of these seven variables are multivriable functions of time $t$ and three spatial variables $x$, $y$, and $z$, and determining what these functions are requires one to solve a large system of nonlinear partial differential equations (PDEs) subject to a set of initial and boundary conditions.  It is impractical or perhaps even impossible to find exact solutions to this system of PDEs.  Thus, one relies on numerical algorithms to solve this mathematical problem. 

This example provides us with another important perspective on numerical analysis. The PDEs just described are a mathematical model for the atmoshpere and thus represent a mathematical abstraction of the physical system. It is an abstraction because these PDEs do not incorporate all possible physical detail. For example, they do not include all of the atomic interactions that occur within the atmosphere. The point is that numerical computation may be viewed as a process for taking mathematical abstractions, *e.g.*, mathematical models, and producing concrete numerical results, and it is important to analyze in a careful manner the consequences of such a process.    

Let us consider a couple of more example problems for those that remain unsure of the role and value of numerical analysis. Consider the problem of finding a solution to the equation $f(x)=0$, where $y=f(x)$ is a polynomial function. If $f$ is a quadratic function, the problem has a solution by way of the well-known quadratic formula. At least two issues arise, one is that the solution may be an [irrational number](https://en.wikipedia.org/wiki/Irrational_number) which can not be represented exactly on a computer. (If you find this statement confusing do not be concerned as we will discuss this issue in detail later.) Another issue is that the algebraic theory of polynomials (known as [Galois theory](https://en.wikipedia.org/wiki/Galois_theory))  tells us that in general there is no analog to the quadratic formula for polynomials of arbitratry degree. Thus, if we want to solve $f(x)=0$ for a general polynomial $f$ on emust result to numerical methods. 

Finally, consider the problem of computing a definite integral 


$$\int_{a}^{b}f(x)\ dx,$$

where $f$ is a continuous function. If $f$ has an anti-derivative $F$ then we may (at least in principle) evaluate the integral by way of the fundamental theorem of calculus. As a specific example, suppose we want to evaluate

$$\int_{1}^{2}e^{-\frac{x^2}{2}}\ dx.$$

Definite integrals of the function $f(x)=e^{-\frac{x^2}{2}}$ appear routinely in probability and statistics. It can be proven that  $f(x)=e^{-\frac{x^2}{2}}$ does not possess an antiderivative. In light of this, we must resort to a numerical method to solve this problem. Of course, our numerical answer will not be exact. Even if an antiderivative is known, even evaluating function values $F(a)$ or $F(b)$ may return irrational numbers. Thus approximation is inevitable. What are the consequences of approximating an abstract mathematical value by a concrete numerical value? Such questions are only understandable within the framework provided to us by numerical analysis. 

Hopefully we have provided some motivation for the study of numerical analysis. Regardless, we have said enough and now it is time to do some mathematics. 

# A First Numerical Method

As a simple case study, consider the problem of approximating the square root of a number $a$. If $a$
is a perfect square, *e.g.*, if $a=4$, then there is not much to worry about. However, if $a$ is not a perfect square, then the true answer may be an [irrational number](https://en.wikipedia.org/wiki/Irrational_number). For example, if $a=2$, then as a real number $\sqrt{2}$ is irrational. (A homework exercise guides you through a simple proof that $\sqrt{2}$ is irrational.) While irrational numbers are well understood as an abtract mathematical concept, they can not be represented exactly on a computer because infinite nature of their decimal expansion. However, any irrational number can be approximated to an arbitratry degree of accuracy by a [rational number](https://en.wikipedia.org/wiki/Rational_number). The question that remains is how to find such an approximation in an efficient, algorithmic way? 

We begin with two observations. The first is that finding $\sqrt{a}$ for a given number $a$ is equivalent to finding a solution to the equation $f(x) = 0$, where $f(x) = x^2 - a$. The second observation is that $\sqrt{a}$ may be interpreted as the length of a side of a square with area $a$. Now, if $x$ is an approximation of $\sqrt{a}$, then the rectangle with sides $x$ and $\frac{a}{x}$ also has area $a$ since 


$$x \times \frac{a}{x} = a.$$

<figure>
<img src="figures/squarerootAlgoDiagram.png" id="fig:notes001" width="400" alt="Figure 1: Square root approximation." 
/><figcaption aria-hidden="true">Figure 1: Square root approximation.</figcaption>
</figure>

Now, if $x$ is an over estimate for $\sqrt{a}$, then $\frac{a}{x}$ is an underestimate for $\sqrt{a}$, and vice-versa. Both of these cases are illustrated in Figure 1. Furthermore, it should be apparent from the figure that the midpoint $\frac{1}{2}\left(x + \frac{a}{x} \right)$ of $x$ and $\frac{a}{x}$ is a better approximation to $\sqrt{a}$ than at least one of  $x$ or $\frac{a}{x}$. This suggests an iterative process for generating a sequence of better and better approximations to $\sqrt{a}$. If $x_{n}$
is an approximation to $\sqrt{a}$, then the formula


$$x_{n+1} = \frac{1}{2}\left(x_{n} + \frac{a}{x_{n}} \right),$$


generates a sequence of approximations to $\sqrt{a}$ that should get closer and closer to the true value as $n$ increases. 

Later we will prove in a more general setting that, for an appropriate initial approximation $x_{0}$ to $\sqrt{a}$ this sequence does in fact converge to a solution of $f(x) = 0$, where $f(x) = x^2 - a$. For now, let's just illustrate what is going on via computation using Julia.

We can implement our algorithm formula in Julia as a function using the following code:

In [2]:
f(x,a) = 0.5*(x + a/x);

Let's look at the case when $a=2$. We know that $1 < \sqrt{2} < \frac{3}{2}$ since $1^2=1 < 2 < \frac{9}{4} = \left(\frac{3}{2}\right)^2$, so let's take as an initial guess $x_{0} = 1.25$ and carry out a few iterations using our formula.

In [3]:
x = 1.25;
println("Initial guess ", x)
x = f(x,2);
println("First iteration ", x)
x = f(x,2);
println("Second iteration ", x)
x = f(x,2);
println("Third iteration ", x)

Initial guess 1.25
First iteration 1.425
Second iteration 1.4142543859649122
Third iteration 1.4142135629622978


Now observe that even after only three iterations of our formula we have a close approximation to $\sqrt{2}$:

In [4]:
x^2 # the square of x value after three iterations

2.000000001666517

Our decision to stop after only three iterations is entirely arbitratry, we could have easily carried out more steps in order to get a better approximation. Later we will discuss in detail issues of when to end an iterative process for an approximation algorithm. 

Notice that since our initial approximation $x_{0}=1.25=\frac{5}{4}$ is rational, all subsequent iterations should be rational numbers because only the operations of multiplication, division, and addition are used and all input values are rational. We can print out our third iterate as a rational number in Julia as follows:

In [5]:
Rational(x)

3184525837589653//2251799813685248

In order to assess the accuracy of our approximation, we will look at the [absolute and relative errors](https://en.wikipedia.org/wiki/Approximation_error) of $x^2$ (the square of our approximation to $\sqrt{2}$) and 2. The **absolute error** is $|x_{\text{exact}} - x_{\text{approximate}}|$ and the **relative error** is $\frac{|x_{\text{exact}} - x_{\text{approximate}}|}{|x_{\text{exact}}|}$.  Another useful quantity is the number of **accurate digits** which is defined as

$$\text{accurate digits} = -\log_{10}\left(\frac{|x_{\text{exact}} - x_{\text{approximate}}|}{|x_{\text{exact}}|} \right).$$

Computing these in Julia gives

In [6]:
abs_err = abs(2 - x^2);
rel_err = abs_err/abs(2);
acc_dig = -log10(rel_err);
println("Absolute error ", abs_err)
println("Relative error ", rel_err)
println("Accurate digits ", acc_dig)

Absolute error 1.6665171465035655e-9
Relative error 8.332585732517828e-10
Accurate digits 9.079220209264447


It is useful to examine how the relative errors decrease with the number of iterations. In a homework exercise, you will be asked to carry out a few more iterations and to compute the relative error at each step. 

# Error and its Sources

In scientific computing, error is inevitable. Thus, error is not something to be avoided but one should always be aware of potential sources of error and how to assess (*i.e.*, measure) and control error. The details and techniques of error analysis will emerge as the course progresses. However, there are a few concepts for which it is useful to have some at least vague awareness of from the start. There are several types of error which may limit the accuracy of a numerical calculation, especially when the calculation is performed on a computer:

1) Errors in the mathematical model. Such error is typically due to the fact that the mathematical model is based on simplified assumptions. This type of error is usually intentional and for the most part outside of the scope of numerical analysis.  

2) **Approximation errors**, of which there are two common types, **discretization errors** and **convergence errors**. Discretization errors occur when one approximates a continuous process (*i.e.*, something that involves taking a limit) with a discrete one. For example, one may often approximate a definite integral by a finite sum, after all, a definite integral is defined to be a continuous limit of sums. An exampelk of a convergence error is provided by our method for approximating the square root of a number. This comes about by stopping an iteration after only a finite number of steps. 

3) **Roundoff error**.  The set of all real numbers $\mathbb{R}$ is uncountably infinite and unbounded. Thus, it is typical to represent only a small subset of  $\mathbb{R}$ on a computer. In fact, we can think of a computer as implementing a discretization of  $\mathbb{R}$ which results in a particular type of discretization error known as roundoff error. Since roundoff errors are very common in computing and have a very specific bechavior, it is useful to single then out as a special case onto themselves.  

# What's Next?

In this course, we will cover the following topics in the given order:

  * Floating point arithmetic and roundoff error.  
  * Numerical solutions of linear systems $Ax = b$ when $A$ is a square matrix ($n \times n$) by direct methods. 
  * Numerical solutions of linear systems $Ax = b$ when $A$ is a rectangular matrix ($m \times n$) with $m > n$. Such systems are said to be **overdetermined**. Such problems occur frequently in machine learning and artificial intelligence, especially in association with so-called supervised learning methods such as regression. 
  

* Numerical solutions of linear systems $Ax = b$ when $A$ is a square matrix ($n \times n$) by iterative methods. While these types of problems are of intrinsic interest, they also arise frequently in association with numerical methods for the solution of partial differential equations. 
* Rootfinding for nonlinear equations. That is, solutions to $f(x) = 0$ where $f$ is a continuous function or vector field. 
* Function interpolation. Here the problem is, given a set of data points $(x_{0},y_{0})$, $(x_{1},y_{1})$, $\ldots$, $(x_{n},y_{n})$ find a function $p(x)$ (typically belonging to some specified class of functions) such that $p(x_{i})=y_{i}$ for all $i=0,1,\ldots,n$. 
* Numerical calculus. That is the numerical approximation of derivatives and integrals. 

Along the way, we will also encounter some concepts such as [**conditioning**](https://en.wikipedia.org/wiki/Condition_number) and [**stability**](https://en.wikipedia.org/wiki/Numerical_stability) that are of importance generally throughout all of numerical analysis. 

In preperation for the next lecture, please watch [this video](https://www.youtube.com/watch?v=97Gb9TS3MJs) on floating point numbers and roundoff error. You may also be interested watch [this video](https://www.youtube.com/watch?v=bRM3zrzZYg8) which reviews some basic matrix algebra that you are expected to know from your prerequisite course in linear algebra.